## CUDA

First, let's address the most important question -- what is CUDA? Is it a programming language? A driver? A hardware architecture? CUDA stands for **Compute Unified Device Architecture**. In its first [Programming Guide][cuda-1.0], NVIDIA described it as a new type of hardware and software architecture for performing general-purpose computations on NVIDIA GPUs without having to use a graphics API.

Today, CUDA ecosystem encompases programming, execution, and memory models, a rich collection of GPU-accelerated libraries, a debugger, a profiler, a compiler, a driver, a toolkit, an integrated development environment, extensions to programming languages such as C and Fortran (called CUDA C and CUDA Fortran), and other tools and know-hows. It is not uncommon to hear people say "CUDA Programming" when they refer to programming in CUDA C or CUDA Fortran, GPGPU programming on NVIDIA GPUs, and even GPGPU programming in general.

Going back to the original question, we can say that CUDA is an all-encompassing solution that enables us to program NVIDIA graphics cards the same way we program CPUs.
In this guide, we will review the programming concepts and hardware details which are essential to CUDA Programming. This information will help you better understand CUDA-oriented Python packages.

[cuda-1.0]: https://developer.download.nvidia.com/compute/cuda/1.0/NVIDIA_CUDA_Programming_Guide_1.0.pdf

### Prerequisites and requirements

1. Basic knowledge of:
    * Unix shell
    * C programming language
1. Access to a Linux machine with:
    * NVIDIA GPU
    * CUDA Toolkit
    * NVIDIA C compiler (`nvcc`)
1. Navigate to the `files` directory of this repository and execute:
```
nvcc get_device_arch.cu -o get_device_arch
./get_device_arch
```
You should see a message that looks like this:

    ```
    Detected 1 CUDA Capable device(s)
    Device 0: "Tesla K80"
    Please use: sm_37
    ```
Write down the last line -- we will need it later.

## Hello, CUDA!

Let's start by examining a short program that illustrates basic CUDA syntax.

In [ ]:
%%file hello_cuda.cu
#include <stdio.h>

__global__ void hello_cuda(void) {
    printf("Hello, CUDA!\n");
}

int main() {
    printf("Hello, world!\n");
    
    hello_cuda <<<1, 8>>>();
    cudaDeviceReset();
    return 0;
}

In this program we display text to the screen: one message from CPU, and a few messages from the GPU. Let's save this program to the `hello_cuda.cu` file: the `.cu` extension is commonly used for files that contains CUDA source code. 

To compile the program, we have to determine one parameter whose meaning we will discuss later. Execute the following command and make a note of the first line of the output:

In [ ]:
!nvcc --list-gpu-code

Output may look look something like this:
```
sm_35
sm_37
sm_50
sm_52
sm_53
sm_60
sm_61
sm_62
sm_70
sm_72
sm_75
sm_80
sm_86
```
In this case, we write down `sm_35` and proceed to compiling the program using the NVIDIA compiler like so:

In [ ]:
!nvcc -arch sm_35 hello_cuda.cu -o hello_cuda

`sm` that we use above stands for "Streaming multiprocessor" and we will talk about them later. Now we can execute the program:

In [ ]:
!./hello_cuda

```
Hello, world!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
``` 

Excellent, we see one "Hello, world!" and eight "Hello, CUDA!" messages. Let's have a closer look at the program.

### Program breakdown

**1. CUDA kernel syntax**<br />
The "Hello, CUDA!" program features our first CUDA kernel &mdash; `hello_cuda`:

```c
__global__ void hello_cuda(void) {
    printf("Hello, CUDA!\n");
}
```

It takes no argumets as all it does is display "Hello, CUDA!" to the screen. Overall, it looks very similar to a regular C function that displays a single message. The only difference is the `__global__` qualifier. This qualifier tells the compiler that the `hello_cuda` is not a regular function but a CUDA kernel, which is a function that is called from the CPU but executed on a GPU.

**2. Calling CUDA kernel**<br />
The next interesting part is the way we launch this kernel from the `main` function:

```c
hello_cuda <<<1, 8>>>();
```

The triple angle brackets speficy execution parameters. In this particular example, we tell the compiler to execute `hello_cuda` kernel using 8 threads. We will talk about the meaning of these numbers in the following sections.

**3. Cleaning up**<br />
Finally, we clean up all GPU the resources by calling the `cudaDeviceReset()` function.

### Exercises

1. Remove `cudaDeviceReset` function call from `hello_cuda.cu`, then compile and run the program again.
2. Replace `cudaDeviceReset` function call with a call to `cudaDeviceSynchronize`, then compile and run the program again.
3. Try compiling the program without the `-arch sm_35` flag.

## CUDA Programming model

In the "Hello, CUDA!" example we wrote our first **CUDA kernel** and executed it using eight **CUDA threads**. Both of these are fundamental elements of the **CUDA programming model** that describes how elements of a CUDA program use  the underlying NVIDIA hardware (_i.e._, the graphics card).

**(Some) Elements of the CUDA programming model**
* CUDA kernel
    * Execution space qualifiers
    * Predefined variables. Global index of a thread.
    * Execution parameters
* Data types
* Variables: qualifier, scope, lifespan
* CUDA threads
    * Hierarchy: threads, blocks, grids
    * Management: synchronization, termination
* Elements of NVIDIA GPU device architecture
    * Streaming Multiprocessors (SMs), warps, caches
    * Compute capabilities
    * Memory bandwidths
* Memory
    * Types: register, local, shared, global, constant, pinned, zero-copy
    * Management: allocation, deallocation, transfer
* CUDA streams and events
* Runtime vs Driver API

### CUDA kernel: execution space qualifiers

A CUDA kernel is a special kind of function that is executed on a GPU device. It is written as a sequential program but executed by multiple CUDA threads each running on a single CUDA core. Conceptually, executing a CUDA kernel is similar to executing a multithreaded function on a CPU:

<a href="./figures/multithreaded-code-vs-cuda-kernel.jpg" target="_blank">
<img src="./figures/multithreaded-code-vs-cuda-kernel.jpg" width="700" alt="Involvement of CPU and GPU cores in execution of multithreaded CPU code and CUDA kernel." />
</a>

In the source code, CUDA kernel declaration must be prefixed with an execution space qualifier that instructs CUDA where the function is called and where it is executed. In the "Hello, CUDA" example we saw one such qualifier &ndash; `__global__`, which tells the compiler that the function will be executed on a GPU but called from a CPU (with some exceptions that we omit in this brief explanation for brevity and clarity). There are two other execution space qualifiers, `__device__` and `__host__`:

1. `__device__`: function is executed on and called from a GPU.
2. `__host__`: function is executed on and called from a CPU.

The `__global__` qualifier can not be combined with the other ones, but `__device__` and `__host__` can be specified together. This is useful when one needs the same functionality on the host and device and wants to reduce code duplication.

<a href="./figures/global_device_host.jpg" target="_blank">
<img src="./figures/global_device_host.jpg" width="700" alt="Effect of execution space qualifiers on where the function/kernel can be called and executed." />
</a>

As a quick exercise, let's rewrite the `hello_cuda` kernel so we can call it from CPU and GPU, that is, let's rewrite it as a `__host__ __device__` function:

In [ ]:
%%file hello_cuda_2.cu
#include <stdio.h>

__host__ __device__ void hello_cuda(void) {
#if defined(__CUDA_ARCH__)
   printf("Hello, CUDA!\n");
#else
   printf("Hello, world!\n");
#endif
}

__global__ void helper_kernel(void) {
    hello_cuda();
}

int main() {
    helper_kernel <<<1,8>>>();
    hello_cuda();
    cudaDeviceReset();
    return 0;
}

In this version of the `hello_cuda` kernel we had to use `__CUDA_ARCH__` macro to differentiate CPU and GPU execution environments. We also had to create a "helper" CUDA kernel -- `helper_kernel` -- to call the `hello_cuda` device function, because we can't call the device function directly from a CPU.

Let's compile and execute it:

In [ ]:
!nvcc -arch sm_35 hello_cuda_2.cu -o hello_cuda_2
!./hello_cuda_2

```
Hello, world!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
```

Notice anything out of order? In the function `main`, we call device function before we call the host function. Yet, in the output we see output from the host function first. This happens because **CUDA kernels are asynchronous**, meaning that they return control to main CPU thread immediately and we must take extra steps when we need to ensure proper order of execution. We can achieve this by calling `cudaDeviceSynchronize` function between the calls to device and host functionс:

```c
int main() {
    helper_kernel <<<1,8>>>();
    cudaDeviceSynchronize();
    hello_cuda();
    cudaDeviceReset();
    return 0;
}
```

Update the main function as shown above, recompile the program and execute it. You should see the following output:

```
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, CUDA!
Hello, world!
```

Now, the host function is not called until the device function finished, so messages from the GPU appear before messages from the CPU. Synchronization is an important topic in CUDA programming and we will come back to it once again soon.

### Thread hierarchy and kernel execution parameters  

To spread out work across CUDA threads, we need to know how to differentiate them. To do that, we need to know the CUDA thread hierarchy model.

When we launch a CUDA kernel, spawned CUDA threads are groupped into blocks which colletively form a _grid_:

<a href="./figures/thread_hierarchy.jpg" target="_blank">
<img src="./figures/thread_hierarchy.jpg" width="400" alt="CUDA thread hierarchy model." />
</a>

That's why you may hear about a "grid of a kernel" and "thread blocks". Within this hierarchy:

* All threads in a grid share global memory space.
* Threads in a single block:
    * can synchronize with each other,
    * have access to block-local shared memory.
* Threads in different blocks cannot cooperate.

The number of blocks in a grid and the number of threads in a block are the two parameters controlled by the programmer at the time a CUDA kernel is launched. These are the execution parameters specified in triple angle brackets (`<<< >>>`).

Recall that in the "Hello, CUDA!" program we launched our kernel using `hello_cuda <<<1, 8>>>();` call. Specified parameters, `<<<1, 8>>>`, instruct CUDA to execute the kernel using a single block of 8 threads:

<a href="./figures/hello_cuda_thread_model.jpg" target="_blank">
<img src="./figures/hello_cuda_thread_model.jpg" width="400" alt="'Hello, CUDA' thread hierarchy model." />
</a>

### Exercise
Execute the `hello_cuda` kernel using:

   * 2 blocks and 4 threads per block
   * 4 blocks and 2 threads per block
   * 8 blocks and 1 thread per block

### Block and thread arrangement

Within the CUDA programming model, blocks in a grid and threads in a block can be arranged in three dimensions. So, in a CUDA kernel call:

```c
kernel_name <<< A, B >>> (arguments);
```

`A` can specify not only the number of blocks in a grid but a 3D arrangement of blocks. Similarly, `B` can specify not only the number of threads in a block but a 3D arrangement of threads within a block. For example, a 2$\times$2$\times$1 2D grid of blocks where every block has 2 threads arranged in 1D (1D blocks) can be created in the following way:

```c
// 2D grid, 1D blocks
dim3 A(2, 2); // same as: dim3 A(2, 2, 1);
dim3 B(2);    // same as: dim3 B(2, 1, 1); 
kernel_name <<< A, B >>> (arguments);
```

Here we use a new data struct &mdash; `dim3`. This is a vectors of 3 integers, each of which defaults to `1`. Components of this data structure are `x`, `y`, and `z`.

Here is another example, where we create a grid populated by a single block that contains 8 threads in a 2$\times$2$\times$2 arrangement:

```c
// 1D grid, 3D blocks
dim3 A; // same as dim3 A(1, 1, 1);
dim3 B(2, 2, 2);
kernel_name <<< A, B >>> (arguments);
```
Of course, we could also call the kernel like so:
```c
kernel_name <<< 1, B >>> (arguments);
```
which would yield the same arrangement of blocks and threads.

### Exercise

Update `hello_cuda` kernel call execution parameters so that the grid has:

1. Two blocks with 4 threads per block in 1D arrangement.
2. Two blocks with 4 threads per block in 2$\times$2$\times$1 arrangement.


<details>
    <summary><strong>&#9656;&nbsp;Solutions</strong></summary>
        
1. 
```c
hello_cuda <<< 2, 4 >>> ();
```
2.
```c
dim3 grid(2);
dim3 block(2, 2);
hello_cuda <<< grid, block >>> ();
```
    </details>

### Predefined variables. Global thread index

When CUDA thread executes a kernel, it has access to 4 predefined variables:

1. `gridDim`: grid dimensions (measured in blocks),
2. `blockDim`: block dimensions (measured in threads),
3. `blockIdx`: block position within a grid, and
4. `threadIdx`: thread position within a block.

All of the variables are of a new type `uint3` which is similar to `dim3` except that there are no default values. These variables can be used to calculate _global thread index_ to uniquely identify each thread and, therefore, its workload. In case of 1D arrangement of blocks and threads, global index of a thread can be computed as:

```c
unsigned int idx = threadIdx.x + blockIdx.x * blockDim.x;
```

`idx` is a widely accepted name for the variable that stores global index of a thread. Here is an example of a kernel that is launched with four 1D blocks each having six 1D threads:
<a href="./figures/global_thread_index.jpg" target="_blank">
<img src="./figures/global_thread_index.jpg" width="500" alt="Global thread index: 1D example" />
</a>

### Exercise

Rewrite the `hello_cuda` kernel so that every thread prints its block, index within the block, and its global thread index.


<details>
    <summary><strong>&#9656;&nbsp;Solutions</strong></summary>
        
```c
__host__ __device__ void hello_cuda(void) {
#if defined(__CUDA_ARCH__)
    unsigned int idx = threadIdx.x + blockIdx.x * blockDim.x;
    printf("Hello from block %d, thread %d, global thread id %d\n", blockIdx.x, threadIdx.x, idx);
#else
    printf("Hello, world!\n");
#endif
}
```
</details>
  



To uniquely identify a thread when using two- and/or three-dimensional arrangement of blocks or threads, one can use either global thread indicies along each direction:

```c
unsigned int idx = threadIdx.x + blockIdx.x * blockDim.x;
unsigned int idy = threadIdx.y + blockIdx.y * blockDim.y;
unsigned int idz = threadIdx.z + blockIdx.z * blockDim.z;
```

or compute a single global thread index:
```c
unsigned int blockDim_xy = blockDim.x * blockDim.y;
unsigned int blockDim_xyz = blockDim_xy * blockDim.z;
unsigned int gridDim_xy = gridDim.x * gridDim.y;
unsigned int idx_local = threadIdx.x  + threadIdx.y * blockDim.x  + threadIdx.z * blockDim_xy;
unsigned int block_shift =  blockIdx.x  +  blockIdx.y *  gridDim.x  +  blockIdx.z *  gridDim_xy;
unsigned int idx = idx_local + block_shift * blockDim_xyz;
```
Here is an example of applying the above rules for computing 1D global thread index:

<a href="./figures/global_thread_index_explained.jpg" target="_blank">
<img src="./figures/global_thread_index_explained.jpg" width="700" alt="Global thread index: 3D example." />
</a>

Keep in mind that there are other ways to calculate global index of a thread: we could traverse blocks and threads in the `z` direction first, or even use different traversal schemes for blocks and threads. However, the best approach takes into account data layout in memory and, typically, matrix-like data are stored linearly with a row-major approach, which corresponds to the traversal approach presented above.


#### Arrangements of blocks and threads - why do we need it?

So, why might we need (or prefer) to use a 2D or 3D arrangement of blocks and threads? The choise of what arrangement of blocks and threads to use is dictated by the input data and convenience. Imagine working with RGB images where every point on a 2D plane provides three values: it is possible to linearize the data into a 1D array and process it using 1D grid of 1D blocks, but it's far more convenient to use 2 separate indicies to uniquely identify each point. The same applies to many other types of data -- 3D distibutions of potentials, vector fields,  and so on.

### Exercise

Modify the `hello_cuda` kernel so that each thread prints the block it belongs to, its thread ID, and global thread ID.

<details>
    <summary><strong>&#9656;&nbsp;Solution</strong></summary>
        
```c
#include <stdio.h>

__host__ __device__ void hello_cuda(void) {
#if defined(__CUDA_ARCH__)
    unsigned int blockDim_xy = blockDim.x * blockDim.y;
    unsigned int blockDim_xyz = blockDim_xy * blockDim.z;
    unsigned int gridDim_xy = gridDim.x * gridDim.y;
    unsigned int idx_local = threadIdx.x  + threadIdx.y * blockDim.x  + threadIdx.z * blockDim_xy;
    unsigned int block_shift =  blockIdx.x  +  blockIdx.y *  gridDim.x  +  blockIdx.z *  gridDim_xy;
    unsigned int idx = idx_local + block_shift * blockDim_xyz;
    printf("blockIdx: (%d, %d, %d)  threadIdx: (%d, %d, %d)  global index: %d\n",
            blockIdx.x, blockIdx.y, blockIdx.z,
            threadIdx.x, threadIdx.y, threadIdx.z,
            idx);
#else
    printf("Hello, world!\n");
#endif
}

__global__ void helper_kernel(void) {
    hello_cuda();
}

int main() {
    dim3 grid(1, 2, 3);
    dim3 block(1, 2, 3);
    helper_kernel <<< grid, block>>>();
    cudaDeviceSynchronize();
    hello_cuda();
    cudaDeviceReset();
    return 0;
}
```
</details>

### Data / Memory Management

So far, we've been developing a naive application that merely prints text to the screen. Understandably, one may want to use GPUs for something more useful, for example, to process or analyze data. Here we have to step back and remind ourselves that CPU-GPU systems are heterogeneous and both CPU and GPU can access their own memory only. This means that in order to process data on the GPU, we have to  transfer it there. Of course, to view or save the results we have to transfer them back to the CPU.

CUDA provides two sister functions for transferring data between the host and device: `cudaMemcpy` and `cudaMemcpyAsync`. They work exactly the same way but the latter one is asynchronous and is used in advanced applications when data throughput is of critical importance.

Another two important data management functions are:

* `cudaMalloc`, which allocates GPU memory, and
* `cudaFree`, which releases allocated GPU memory. 

So let's talk about how these functions enable data analysis on GPU. First, let's have a look at the (simplified) signature of `cudaMemcpy`:

```c
cudaMemcpy(*destination, *source, nbytes, direction);
```

Arguments of the `cudaMemcpy` command are:

- `*destination`: pointer to the memory location where the data is sent (copied) to
- `*source`: pointer to the data being transferred.
- `nbytes`: transferred data size in bytes,
- `direction`: direction of memory transfer. Possible values are:
    * `cudaMemcpyHostToDevice`
    * `cudaMemcpyDeviceToHost`
    * `cudaMemcpyDeviceToDevice`
    * `cudaMemcpyHostToHost`

### Four steps of memory management

In general, processing data on a GPU device has six major steps, four of which deal with memory management:

**Step 1**: Allocate memory on the device (`cudaMalloc`) and host (`malloc`).<br>
_Prepare data on the host (initialize, read, generate, etc)_<br>
**Step 2**: Transfer data from host to device (`cudaMemcpy`, direction: `cudaMemcpyHostToDevice`)<br>
_Process data on the device_<br>
**Step 3**. Transfer the results back to host (`cudaMemcpy`, direction: `cudaMemcpyDeviceToHost`)<br>
**Step 4**. Free memory on the device (`cudaFree`) and host (`free`).

Here is an example:

```c
// define data parameters
unsigned int arr_size = 1<<22; 
unsigned int nbytes = arr_size * sizeof(float);

// Allocate host memory
float *h_a = (float *)malloc(nbytes);

// Allocate device memory
float *d_a;
cudaMalloc((float**)&d_a, nbytes);

// Initialize host memory
for (unsigned int i=0; i<arr_size; i++) h_a[i] = 0.5f * i;

// Transfer data from host to device
//          to<~from                  from~>to
cudaMemcpy(d_a, h_a, nbytes, cudaMemcpyHostToDevice);

// Process data on the device

// Transfer data back from device to host
//          to<~from                     from~>to
cudaMemcpy(h_a, d_a, nbytes, cudaMemcpyDeviceToHost);

// free memory
cudaFree(d_a);
free(h_a);
```

### Exercise

Write a program that multiplies an array of floating point numbers by 2.

<details>
    <summary><strong>&#9656;&nbsp;Solution</strong></summary>
    
```c
#include <stdio.h>

__global__
void multiply(float *in, float *out, const int n) {
    unsigned int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n)
        out[idx] = in[idx] * 2.0f;
}

int main() {
    unsigned int arr_size = 1<<22; 
    unsigned int nbytes = arr_size * sizeof(float);
    
    // Allocate host memory
    float *h_in = (float *)malloc(nbytes);
    float *h_out = (float *)malloc(nbytes);
    
    // Allocate device memory
    float *d_in;
    float *d_out;
    cudaMalloc((float **)&d_in, nbytes);
    cudaMalloc((float **)&d_out, nbytes);

    // Initialize host memory
    for (unsigned int i=0; i<arr_size; i++)
        h_in[i] = 0.5f * i;

    // Transfer data from host to device
    //         to <~ from                    from~>to
    cudaMemcpy(d_in, h_in, nbytes, cudaMemcpyHostToDevice);
    
    // Set kernel execution parameters: 
    // 1. Use 128 threads per block
    // 2. Compute the number of necessary blocks
    int blocksize = 128;
    int gridsize = (arr_size + blocksize - 1) / blocksize;

    dim3 grid(gridsize);
    dim3 block(blocksize);

    multiply <<<grid, block>>>(d_in, d_out, arr_size);
    
    // Transfer data back from device to host
    //          to <~ from                     from~>to
    cudaMemcpy(h_out, d_out, nbytes, cudaMemcpyDeviceToHost);
    
    // test results
    for (unsigned int i=1; i<arr_size; i = i<<1)
        printf("h_out[%d] = %f\n", i, h_out[i]);

    // release memory
    cudaFree(d_in);
    cudaFree(d_out);
    free(h_in);
    free(h_out);
    
    cudaDeviceReset();
    return 0;
}
```
</details>



### Memory bandwidths in a CPU-GPU system

Efficient memory transfer is an important part of every CUDA application that deals with data. This is due to the intrinsic limitations of the hardware components of heterogeneous CPU--GPU systems. The following diagram shows important memory paths in a CPU-GPU heterogeneous system: 

<a href="./figures/memory_bandwidth.jpg" target="_blank">
<img src="./figures/memory_bandwidth.jpg" width="600" />
</a>

PCIe is the slowest path in a heterogeneous system and as a programmers you should always try to minimize the amount of data transferred using this path or overlap such transfers with other useful work.

### CUDA Memory model

Memory access and management are important aspects of the CUDA programming model because improper or inefficient use of memory may lead to degraded performance and bottlenecks. CUDA memory model exposes many types of GPU memory:

* Registers
* Local memory
* Shared memory
* Constant memory
* Texture memory
* Global memory

<a href="./figures/memory_hierarchy.jpg" target="_blank">
<img src="./figures/memory_hierarchy.jpg" width="600" alt="CUDA memory model" />
</a>
    
Each type of memory has its own scope, lifetime, and purpose:
- **Registers**: store a limited amount of automatic variables that are declared in a kernel without special qualifiers. Fastest memory space on a GPU. Fermi cards have 63 registers per thread, Kepler cards - 255 registers per thread. Has a lifetime of a kernel (thread).
   Example:
   ```
   float pi = 3.14f;
   ```
- **Local memory**: for variables declared in a kernel that don't fit into registers. Each thread has its own private local memory. Name is misleading: "local memory" resides in the same physical location as Global memory. Has a lifetime of a kernel (thread).
   Example:
   ```
   float data[100];
   ```
- **Shared memory**: memory within a thread block that is visible to all threads and whose lifetime is the lifetime of the block. Faster (higher bandwidth, lower latency) than local or global memory. Access to shared memory must be synchronized using `__syncthreads()` function that creates a barrier that all threads in the same thread block must reach before any other thread is allowed to proceed. Variables decorated with `__shared__` are stored in shared memory.
   Example:
   ```
   __shared__ float var;
   __shared__ float data[100];
   ```
- **Global memory**: Largest, highest-latency memory that all threads can access. The name _global_ refers to its scope (global) and lifetime (application). Dynamic allocation/deallocation _via_ `cudaMalloc` and `cudaFree` host functions. Variables decorated with `__device__` are stored in the global memory.
    ```
    __device__ float var;
    __device__ float data[100];
    ```
    Note that to transfer statically declared variables to/from GPU, one has to use `cudaMemcpyToSymbol` and `cudaMemcpyFromSymbol` functions:
    ```
    // file-scope
    __device__ float devVar;
    
    // in main()
    float value = 3.14f;
    cudaMemcpyToSymbol(devVar, &value, sizeof(float));
    ...
    cudaMemcpyFromSymbol(&value, devData, sizeof(float));
    ```
- **Constant memory** is read-only memory space for statically declared variables (no `cudaMalloc` etc) that are visible to all kernels. Kernels can only read from the constant memory. Must be declared outside of any kernels. Good use case: a coefficient or a constant for a mathematical formula. Not so good use case: when each thread (in a _warp_) reads its own address once.
    Example:
    ```
    __constant__ float var;
    __constant__ float data[100];
    ```
- **Texture memory**: special kind of read-only global memory that can perform floating-point interpolation as part of the read process ("filtering").

Besides the above types of GPU memory, CUDA memory model also exposes host and hybrid types of memory:

- **Pinned memory** is a host memory that the host operating system can not move to another physical location. Also called _page-locked memory_. Allocated on host via `cudaMallocHost` and freed with `cudaFreeHost`. More "expensive" to allocate and deallocate, but provides better throuput. Beneficial for sizeable data transfers (10MB of data).
- **Zero-copy memory**: pinned host memory that is mapped into the device address space. Both the host and device can access zero-copy memory but memory accesses must be synchronized. OK to use for small amount of data because it simplifies programming. For large datasets, zero-copy memory is a poor choice because it causes significant performance degradation.
- **Unified memory**: a pool of managed memory, where each allocation is accessible on both the CPU and GPU with the same memory address. The underlying system automatically migrates data between host and device when necessary. Like _zero-copy memory_, offers a "single pointer to data" model but transparently migrates data between host and device as necessary to improve performance.
    Example:
    ```
    __device__ __managed__ float data; // in file-scope or global-scope

    cudaMallocManaged(void **devhostPtr, size_t size, int flags=0);
    ```

## CUDA Streams

All CUDA operations, whether explicitly or implicitly, run within the so-called CUDA streams. A CUDA stream is a sequence of CUDA operations executed on a device in the order prescribed by the host code. When CUDA stream is not specified, CUDA operations are performed in the default stream called the NULL stream.

CUDA streams enable concurrent execution of multiple independent sequences of CUDA operations, thus improving utilization of the underlying hardware. CUDA streams can be compared to railroad tracks that provide a pathway for the trains &mdash; CUDA operations (kernels, data transfers, etc).

CUDA streams can be blocking and non-blocking. Blocking streams can not run in parallel with the default NULL stream but can run in parallel with each other. In the railroads-trains analogy, blocking streams are the railroads that share a common power source with the default NULL stream and non-blocking streams have their own power source.

Data transfer operations may not always be executed in parallel even when they're issued in different streams. This is due to the fact that they must share a common resource - the PCIe bus. Only data transfers in opposite directions and in different streams can run in parallel with each other. In the railroads-trains analogy, data transfers can be compared to tunnels with two tracks running in opposite directions and trains that are going in the same direction must share a single railroad track.

<a href="./figures/streams_railroad_analogy.jpg" target="_blank">
<img src="./figures/streams_railroad_analogy.jpg" alt="Railroad tracks analogy for CUDA streams" />
</a>

### CUDA Stream commands

To create CUDA streams, we first need to initialize variables that we will use to refer to them:

```
// single stream
cudaStream_t stream;

// multiple streams
cudaStream_t *streams = (cudaStream_t *)malloc(n_streams * sizeof(cudaStream_t));
```

Then, we can create regular blocking streams and non-blocking streams like so:
```
// blocking stream
cudaStreamCreate(&stream); 

// non-blocking stream
cudaStreamCreateWithFlags(&stream, cudaStreamNonBlocking);
```

To release resources of a stream (to destroy a stream):
```
cudaStreamDestroy(stream);
```

Because all CUDA stream operations are asynchronous, it's important to be able to query a stream about its status and to tell CUDA to wait for a stream to complete its operations. CUDA provides two functions for that:

```
// block host until operations in the stream have completed
cudaStreamSynchronize(stream);

//Query the stream about its status. 
cudaStreamQuery(stream)
```

To do memory transfer within a stream, we have to use `cudaMemcpyAsync` function:
```
cudaMemcpyAsync(destination, source, shared_memory_size, stream);
```

CUDA streams come hand in hand with CUDA events, which are essentially markers of progress in CUDA streams. They, however, fall beyond the scope of this introduction to CUDA and we'd like to encourage our students to read about them in the referenced material.

### NVIDIA GPU architecture overview

Now let's discuss the hardware and how it affects CUDA applications. NVIDIA graphics card that support CUDA are called CUDA-capable or CUDA-enabled. There are several generations of such cards and each generation brings new features and capabilities: ability to display text right from the kernel (i.e., use `printf` function), dynamic parallelism, unified memory, and so on. These capabilities are called _compute capabilities_ and NVIDIA assignes them version numbers, also sometimes called "SM versions". This number is used by applications at runtime to determine which hardware features are available on the present GPU device. More information about compute capabilities you can find here: https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities

### SM: Streaming Multiprocessor

As we mentioned above, compute capability version is also called "SM version". SM stands for Streaming Multiprocessor, which is the fundamental component of a CUDA-enabled NVIDIA GPU. Here is a diagram of one of such GPUs (Kepler K20x):

<a href="./figures/kepler_k20x.jpg" target="_blank">
<img src="./figures/kepler_k20x.jpg" alt="Diagram of a CUDA-enabled NVIDIA GPU (Kepler K20x)." />
</a>

As you can see, NVIDIA GPU architecture is built around Streaming Multiprocessors that house all the computing resources: CUDA cores, Load/Store units, Special Function Units, Register file, Shared memory, warps, and etc. Parallelism is achieved by replicating Streaming Multipricessors, so a more powerful GPU can be constructed by increasing the number of these SMs.

### SMs, kernels, and grids

When a kernel grid is launched, the thread blocks of that kernel grid are distributed among available SMs for execution. Once _scheduled_ on an SM, threads of a thread block _execute_ concurrently only on that assigned SM. Multiple thread blocks may be assigned to the same SM at once and are scheduled based on the availability of SM resources (there is a limit on the number of threads an SM can run concurrently).

### Warps and SIMT

CUDA employes a Signle Instruction Multiple Thread (SIMT) architecture to manage and execute threads in groups of 32 called warps. All threads in a warp execute the same instruction at the same time. Each SM partitions the thread blocks assigned to it into 32-thread warps that it then schedules for execution on available hardware resources. The SIMT architecture is similar to SIMD except that SIMT allows multiple threads in the same warp to execute independently, whereas SIMD requires synchronous execution.


### Magic number: 32

As we mentioned above, warps are groups of 32 threads. 32 is a magic number in CUDA programming. Optimiting your workloads to fit within the boundaries of a warp will generally lead to better utilization of GPU resources and, therefore, better performance of your application. That's why programmers often use blocks of 32, 64, 128, and 1024 threads. Note that there is a limit on the number of threads per block -- 1024, which corresponds to 32 warps.

## References

1. CUDA Programming Guide: https://docs.nvidia.com/cuda
2. "Professional CUDA C Programming" by John Cheng, Max Grossman, Ty McKercher. ([wiley.com](https://www.wiley.com/en-us/Professional+CUDA+C+Programming-p-9781118739310), [amazon.com](https://www.amazon.com/dp/1118739329/ref=cm_sw_em_r_mt_dp_GEF22X8H3JJKS51G5107))
3. "GPU Parallel Program Development Using CUDA" by Tolga Soyata ([routledge.com](https://www.routledge.com/GPU-Parallel-Program-Development-Using-CUDA/Soyata/p/book/9780367572242), [amazon.com](https://www.amazon.com/dp/1498750753/ref=cm_sw_em_r_mt_dp_5KACSGQMH4RZBJG988GB))